In [1]:
import sys
import pathlib
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import dimod
import dwave
import dwave.system
from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
import dwave.inspector
import dwave_networkx as dnx
import minorminer
import embera

from src.particle_funcs import distance_matrix as distance_matrix
import src.leap_funcs.qubo.q_matrix as q_matrix

from src import h5py_funcs
from src.h5py_funcs import discoveries, init_custom_getstates, io

In [2]:
read_embedding = h5py_funcs.io.read_embedding_from_hdf5_file('test.h5')

Filename:  test.h5
 - composite   <HDF5 group "/composite" (7 members)>
 - embedding   <HDF5 group "/embedding" (25 members)>
 - group   <HDF5 group "/group" (1 members)>
 - group2   <HDF5 group "/group2" (2 members)>
 - group3   <HDF5 group "/group3" (1 members)>
 - group4   <HDF5 group "/group4" (2 members)>
 - particles   <HDF5 group "/particles" (2 members)>
 - sampler   <HDF5 group "/sampler" (10 members)>
 - test   <HDF5 dataset "test": shape (100, 100), type "<f8">

<HDF5 group "/embedding" (25 members)>
25
ItemsViewHDF5(<HDF5 group "/embedding" (25 members)>)


In [3]:
print(type(read_embedding))
print(read_embedding)
print(type([key for key in read_embedding.keys()][0]))
print(type(read_embedding[1]))
print(type(read_embedding[1][0]))

<class 'dict'>
{1: [3996, 3997], 10: [1250, 3951, 3891], 11: [1415, 4027], 12: [1520, 3952], 13: [3922, 1475], 14: [3982, 3981], 15: [1430, 3892], 16: [4012, 1400, 1476], 17: [4101, 4102, 1535], 18: [1461, 1460], 19: [3967, 3966, 1505], 2: [1325, 4086, 4087], 20: [1369, 1370, 3862], 21: [3771, 1384, 1385], 22: [3937, 3936], 23: [3878, 3877, 1580], 24: [3831, 1324, 1280], 25: [3847, 1310], 3: [1445, 1446, 4057], 4: [1340, 1339], 5: [1311, 4042, 4011], 6: [1355, 1354], 7: [1295, 1296, 4072, 4073], 8: [3907, 3906], 9: [1265, 3921]}
<class 'int'>
<class 'list'>
<class 'numpy.int32'>


In [4]:
with open('../API_Token.txt') as file:
    token = file.readline().rstrip()
    architecture = file.readline().rstrip()
sampler = DWaveSampler(token = token, architecture='pegasus', region='eu-central-1')
tmp_can_be_removed_ = sampler.adjacency # required for sampler having all data needed for __getstate__, no idea why this is necessary

In [5]:
num_particles = 5

use_dimod_exact_solver = False

part_coords_n = np.zeros((num_particles,2))
part_coords_nm1 = np.zeros((num_particles,2))

for i in range(np.shape(part_coords_n)[0]):
    part_coords_n[i,:] = [0, i]
    part_coords_nm1[i,:] = [0.5*1, i]

#fig_initial, axs_initial = plt.subplots(1,1)
#axs_initial.scatter(part_coords_n[:,0], part_coords_n[:,1], label="n")
#axs_initial.scatter(part_coords_nm1[:,0], part_coords_nm1[:,1], label="n-1")
#fig_initial.legend()
#fig_initial.show()

correct_sol = np.zeros(num_particles*num_particles)
for i in range(1, num_particles+1):
    correct_sol[(i-1)*num_particles + i -1] = 1.

print(correct_sol)

[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 1.]


In [6]:
distances = distance_matrix.calc_phi_ij(part_coords_n, part_coords_nm1)
Q_dist_diag = q_matrix.q_dist_diag(distances)
Q_part = q_matrix.q_part(np.shape(distances)[0])
Q_pos = q_matrix.q_pos(np.shape(distances)[0])
Q_array = Q_dist_diag + Q_part + Q_pos
Q_dict = q_matrix.Q_convert_to_dict(Q_array)
#with np.printoptions(precision=3, linewidth=1000, threshold=1e6):
#    print(Q_array)

In [7]:
composite_2 = FixedEmbeddingComposite(sampler, read_embedding)
num_qubits_comp_2 = len(set(inner for outer in read_embedding.values() for inner in outer))
print(num_qubits_comp_2)
params = {'label' : 'superdupernice label',
          'annealing_time': 5, 
          'num_reads': 3, 
          'answer_mode': 'raw', 
          'programming_thermalization': 1000, 
          'readout_thermalization': 0}
print(composite_2.child.solver.estimate_qpu_access_time(num_qubits=num_qubits_comp_2, **params))

63
14664.426470404487


In [8]:
samples = composite_2.sample_qubo(Q_dict, **params)

In [9]:
samples

SampleSet(rec.array([([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], -6.26393202, 1, 0.),
           ([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], -5.02786405, 1, 0.),
           ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], -4.76393202, 1, 0.)],
          dtype=[('sample', 'i1', (25,)), ('energy', '<f8'), ('num_occurrences', '<i4'), ('chain_break_fraction', '<f8')]), Variables([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]), {'timing': {'qpu_sampling_time': 279.48, 'qpu_anneal_time_per_sample': 5.0, 'qpu_readout_time_per_sample': 67.62, 'qpu_access_time': 15157.53, 'qpu_access_overhead_time': 2382.47, 'qpu_programming_time': 14878.05, 'qpu_delay_time_per_sample': 20.54, 'total_post_processing_time': 228.0, 'post_processing_overhead_time': 228.0}, 'problem_id': '5eb050cb-1f5e-48d1-815f-c6bccf6f85a6', 'problem_label': 'superdupernice label', 'embeddi

In [10]:
from src.leap_funcs.qubo import filter_samples

ids_valid_solutions= filter_samples.indices_of_valid_solutions(samples, sort_energies=False)
print(ids_valid_solutions)
print(samples.record[ids_valid_solutions])

[0, 1]
[([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], -6.26393202, 1, 0.)
 ([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], -5.02786405, 1, 0.)]


In [11]:
for i, sol in enumerate(samples.record):
    if i in ids_valid_solutions:
        if np.array_equal(correct_sol, sol[0]):
            print('sol', i, 'is valid and optimal')
            print(' ', sol)
        else: 
            print('sol', i, 'is valid but not optimal')
            print(' ', sol)
for i, sol in enumerate(samples.record[ids_valid_solutions]):
    if np.array_equal(correct_sol, sol[0]):
        print(ids_valid_solutions[i])
        print(sol)

sol 0 is valid but not optimal
  ([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], -6.26393202, 1, 0.)
sol 1 is valid but not optimal
  ([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], -5.02786405, 1, 0.)
